In [44]:
from seleniumwire import webdriver
import json
#from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime
import requests
import pandas as pd

In [45]:
path = "C:\\Users\\PATH TO YOUR CHROMEDRIVER\\chromedriver.exe"
website = "https://inloggen.eneco.nl/"


In [46]:
options = webdriver.ChromeOptions()
options.set_capability("goog:loggingPrefs", {'performance': 'ALL'})
driver = webdriver.Chrome(executable_path=path, chrome_options=options)

driver.implicitly_wait(20)
driver.get(website)

In [47]:
try: #catches non connection to internet of WA
        time.sleep(5)
        inlog = WebDriverWait(driver,40).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input#okta-signin-username")))
        inlog.send_keys('your e-mail')
except:
    driver.save_screenshot('fail_login.png')
    driver.quit()
    print("fail")

In [48]:
try: #catches non connection to internet of WA
        time.sleep(5)
        btn = WebDriverWait(driver,40).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input#okta-signin-submit")))
        btn.click()
except:
    driver.save_screenshot('fail_button.png')
    driver.quit()
    print("fail")

In [49]:
try: #catches non connection to internet of WA
        time.sleep(5)
        inlog = WebDriverWait(driver,40).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input#input52")))
        inlog.send_keys('YOUR PASSWORD')
except:
    driver.save_screenshot('fail_pwd.png')
    driver.quit()
    print("fail")

In [50]:
try: #catches non connection to internet of WA
        time.sleep(5)
        btn = WebDriverWait(driver,40).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input.button.button-primary")))
        btn.click()
except:
    driver.save_screenshot('fail_submit.png')
    driver.quit()
    print("fail")

In [51]:
try: #catches non connection to internet of WA
        time.sleep(5)
        btn = WebDriverWait(driver,40).until(EC.presence_of_element_located((By.CSS_SELECTOR,"button.c-diyrn.c-PJLV.c-diyrn-gsLTSO-size-regular.c-diyrn-hslrZy-cv.sparky-button")))
        btn.click()
except:
    driver.save_screenshot('fail_cookie.png')
    driver.quit()
    print("fail")

In [52]:
del driver.requests

In [53]:
driver.get('https://www.eneco.nl/mijn-eneco/verbruik')
time.sleep(5)

In [54]:
for r in driver.iter_requests():
    if str(r).find('usages') > 0:
        #print(r.headers['Authorization'])
        au = r.headers['authorization']
        apikey = r.headers['apikey']       


In [55]:
driver.quit()

In [56]:
r=requests.get("https://api-digital.enecogroup.com/dxpweb/nl/eneco/customers/41128821/accounts/2/usages?aggregation=Day&interval=Hour&start=2022-12-25&end=2023-01-03&addBudget=false&addWeather=true&extrapolate=false", headers={"authorization":au, "apikey":apikey}, verify=False)
result = r.content

c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-digital.enecogroup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [57]:
r = json.loads(result)

In [58]:
df = pd.DataFrame(r['data']['usages'])

In [59]:
dates = []
usages = []

for entry in df['entries']:
    for a in entry:    
        dates.append(a['actual']['date'])
        usages.append(a['actual']['totalUsageCostInclVat'])

In [60]:
result = pd.concat([pd.DataFrame(dates), pd.DataFrame(usages)], axis=1)